In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'UOW/AISecurity/assignment1/'
FOLDERNAME = 'UOW/998Image2Text/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/My Drive/{}/codebase'.format(FOLDERNAME))

%cd /content

Mounted at /content/drive
/content


In [ ]:
%cd "/content/drive/My Drive/UOW/998Image2Text"
!ls

/content/drive/My Drive/UOW/998Image2Text
 bpe_simple_vocab_16e6.txt.gz	       '狗 (1).jpg'    '飞机 (2).jpg'
 clip-imp-pretrained_128_6_after_4.pt  '狗 (2).jpg'    '飞机 (3).jpg'
 custom_clip.py			       '狗 (3).jpg'    '飞机 (4).jpg'
 custom_model.py		       '狗 (4).jpg'     飞机.jpg
 custom_tokenizer.py		        狗.jpg
 __pycache__			       '飞机 (1).jpg'


In [ ]:
import torch
import sys

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

print('PyTorch Version:', torch.__version__)
print('-' * 60)
if torch.cuda.is_available():
    print('CUDA Device Count:', torch.cuda.device_count())
    print('CUDA Device Name:')
    for i in range(torch.cuda.device_count()):
        print('\t', torch.cuda.get_device_name(i))
    print('CUDA Current Device Index:', torch.cuda.current_device())
    print('-' * 60)

print(f"Python version = {sys.version}")

PyTorch Version: 2.6.0+cu124
------------------------------------------------------------
Python version = 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [ ]:
# 安装依赖
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

import torch
import clip
from PIL import Image
import matplotlib.pyplot as plt

# 加载模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 上传两张图像（狗和飞机）——你可以在Colab点击左边“文件”图标上传
from google.colab import files
uploaded = files.upload()

# 读取图像并预处理
image_paths = list(uploaded.keys())
images = [preprocess(Image.open(p)).unsqueeze(0).to(device) for p in image_paths]
images = torch.cat(images)

# 文本输入
texts = clip.tokenize(["This is a dog.", "This is an airplane."]).to(device)

# 获取图像和文本的特征
with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(texts)

# 计算相似度（余弦相似度）
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T)

# 显示匹配结果
for i, path in enumerate(image_paths):
    print(f"Image: {path}")
    for j, text in enumerate(["This is a dog.", "This is an airplane."]):
        print(f"  '{text}': {similarity[i][j].item():.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7nupvykf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7nupvykf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 92.3MiB/s]


Saving 狗.jpg to 狗.jpg
Saving 飞机.jpg to 飞机.jpg
Image: 狗.jpg
  'This is a dog.': 25.62
  'This is an airplane.': 20.24
Image: 飞机.jpg
  'This is a dog.': 18.08
  'This is an airplane.': 26.36


In [ ]:
# 下载 clip.py 和 model_loader.py
!wget https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/clip.py
!wget https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/model_loader.py
!wget https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/model.py
!wget https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/simple_tokenizer.py


--2025-07-26 07:47:18--  https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/clip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7489 (7.3K) [text/plain]
Saving to: ‘clip.py’

clip.py             100%[===================>]   7.31K  --.-KB/s    in 0.001s  

2025-07-26 07:47:18 (5.37 MB/s) - ‘clip.py’ saved [7489/7489]

--2025-07-26 07:47:18--  https://raw.githubusercontent.com/rajpurkarlab/CXR-RePaiR/main/clip/model_loader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-26 07:47:18 ERROR 404: Not Found

In [ ]:
!mv clip.py custom_clip.py
!mv model.py custom_model.py
!mv simple_tokenizer.py custom_tokenizer.py

mv: cannot stat 'clip.py': No such file or directory
mv: cannot stat 'model.py': No such file or directory
mv: cannot stat 'simple_tokenizer.py': No such file or directory


In [ ]:
!wget https://openaipublic.blob.core.windows.net/clip/bpe_simple_vocab_16e6.txt.gz -O /content/drive/MyDrive/UOW/998Image2Text/bpe_simple_vocab_16e6.txt.gz
vocab_path = "/content/drive/MyDrive/UOW/998Image2Text/bpe_simple_vocab_16e6.txt.gz"


--2025-07-26 07:46:36--  https://openaipublic.blob.core.windows.net/clip/bpe_simple_vocab_16e6.txt.gz
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.104.161
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.104.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1356917 (1.3M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/UOW/998Image2Text/bpe_simple_vocab_16e6.txt.gz’

/content/drive/MyDr 100%[===================>]   1.29M  3.78MB/s    in 0.3s    

2025-07-26 07:46:37 (3.78 MB/s) - ‘/content/drive/MyDrive/UOW/998Image2Text/bpe_simple_vocab_16e6.txt.gz’ saved [1356917/1356917]



In [ ]:
!pip install ftfy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


In [ ]:
from custom_clip import load  # custom_clip.py 中的load()函数会自动构建并返回模型

model, _ = load(name="ViT-B/32", device="cpu", jit=False)

# 或者你可以传入参数后调用 model.encode_image() 等接口


100%|███████████████████████████████████████| 354M/354M [00:05<00:00, 69.6MiB/s]


In [ ]:
# 加载训练好的 .pt 文件参数
import torch

model_path = "/content/drive/MyDrive/UOW/998Image2Text/clip-imp-pretrained_128_6_after_4.pt"
state_dict = torch.load(model_path, map_location="cpu")
model.load_state_dict(state_dict)
model.eval()  # 设置为推理模式


CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# ✅ 第一步：导入包
import torch
from PIL import Image
import torchvision.transforms as T
from custom_clip import load
from custom_tokenizer import SimpleTokenizer
import os

# ✅ 第二步：加载模型结构 & 权重
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _ = load(name="ViT-B/32", device=device, jit=False)

# 加载你训练的模型参数
model_path = "/content/drive/MyDrive/UOW/998Image2Text/clip-imp-pretrained_128_6_after_4.pt"
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

# ✅ 第三步：定义图像预处理方式
preprocess = T.Compose([
    T.Resize(224, interpolation=Image.BICUBIC),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]),
])

# ✅ 第四步：上传并处理图像
from google.colab import files
uploaded = files.upload()
image_paths = list(uploaded.keys())

images = [preprocess(Image.open(p).convert("RGB")).unsqueeze(0).to(device) for p in image_paths]
images = torch.cat(images)  # [N, 3, 224, 224]

# ✅ 第五步：处理文本
texts = ["This is a dog.", "This is an airplane."]
tokenizer = SimpleTokenizer()
tokenized = [tokenizer.encode(t) for t in texts]

# 把token列表变成张量，pad到同样长度
# max_len = max(len(t) for t in tokenized)
max_len = 77
input_text_tensor = torch.zeros(len(texts), max_len, dtype=torch.long).to(device)
for i, tokens in enumerate(tokenized):
    input_text_tensor[i, :len(tokens)] = torch.tensor(tokens)

# 打印，确认 pad 成功
print("input_text_tensor shape (should be [2, 77]):", input_text_tensor.shape)



# ✅ 第六步：提取特征
with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(input_text_tensor)

# ✅ 第七步：计算相似度
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T)

# ✅ 第八步：输出结果
for i, path in enumerate(image_paths):
    print(f"\nImage: {path}")
    for j, text in enumerate(texts):
        print(f"  '{text}': {similarity[i][j].item():.2f}")


Saving 狗.jpg to 狗 (3).jpg
Saving 飞机.jpg to 飞机 (1).jpg
input_text_tensor shape (should be [2, 77]): torch.Size([2, 77])

Image: 狗 (3).jpg
  'This is a dog.': 28.22
  'This is an airplane.': 25.43

Image: 飞机 (1).jpg
  'This is a dog.': 20.38
  'This is an airplane.': 24.84
